# **Webscraping de marmiton**

# **1. Installation de Selenium**
Pour mimer le comportement d'un utilisateur qui navigue sur une page web (et pas sur une page statique), on utilise Selenium.
Cela nécessite l'installation de quelques dépendances avant l'installation de Selenium lui-même.

## 1.1. On installe les dépendances

In [1]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb
!sudo apt-get update
!sudo -E apt-get install -y /tmp/chrome.deb
!pip install chromedriver-autoinstaller selenium

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

--2024-01-08 16:54:16--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.201.174, 2a00:1450:4007:81a::200e
Connecting to dl.google.com (dl.google.com)|142.250.201.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105001444 (100M) [application/x-debian-package]
Saving to: ‘/tmp/chrome.deb’

/tmp/chrome.deb     100%[===================>] 100.14M  58.7MB/s    in 1.7s    

2024-01-08 16:54:18 (58.7 MB/s) - ‘/tmp/chrome.deb’ saved [105001444/105001444]

Get:1 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://apt.postgresql.org/pub/repos/apt jammy-pgdg InRelease [123 kB]   
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]        
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd6

'/opt/mamba/lib/python3.10/site-packages/chromedriver_autoinstaller/120/chromedriver'

## 1.2. On installe Selenium

In [3]:
import selenium
from webdriver_manager.chrome import ChromeDriverManager

path_to_web_driver = ChromeDriverManager().install()

# **2. Lancement de marmiton**

## 2.1. On initialise le navigateur
Pour mimer le comportement d'un utilisateur, on doit initialiser le navigateur, notammment pour pouvoir mimer un temps de latence (un utilisateur n'agit pas de manière absolument instantanée) et le fait d'actionner les touches du clavier.

In [4]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## 2.2. On lance le navigateur
On exécute le navigateur comme le ferait un utilisateur.

In [5]:
from selenium.webdriver.chrome.service import Service
service = Service(executable_path=path_to_web_driver)

browser = webdriver.Chrome(service=service,
                           options=chrome_options)

## 2.3. On se rend sur marmiton
La commande "get" nous permet de réaliser cette action très facilement. 

In [6]:
browser.get('https://www.marmiton.org')
print('Arrivée sur marmiton')

Arrivée sur marmiton


# **3. Gestion des cookies**
Lorsqu'on arrive sur le site de Marmiton, un popup s'ouvre qui demande à ce que l'on accepte les cookies.\
Pour que cette acceptation se fasse automatiquement, on recherche dans le code source le bouton "accepter les cookies" et on mime le comportement d'un utilisateur qui actionnerait cette commande.

In [7]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException

try : 
    popup = browser.find_element(By.ID, "didomi-notice-agree-button")
    popup.click()
    time.sleep(2)
except NoSuchElementException:
    pass

print("Entrée sur marmiton réussie \n")

Entrée sur marmiton réussie 



# **4. Recherche de recettes pour un ingrédient donné**

## 4.1. On recherche l'ingrédient choisi dans la barre de recherche du site
a) On se place dans la barre de recherche\
b) On mime le comportement d'un utilisateur qui utiliserait le clavier pour formuler sa requête (en terminant par actionner la touche "ENTREE"/"RETURN")

In [8]:
search = browser.find_element("name", "aqt")
print(search)

search.send_keys("farine")
search.send_keys(Keys.RETURN)

time.sleep(1)

print("recettes trouvées")

<selenium.webdriver.remote.webelement.WebElement (session="a69c6c4e9bdbbe4fa271041b3f5059ca", element="FF3A66C5F13BBB3D9A26039BA9692BF7_element_20")>
recettes trouvées


## 4.2. On résupère l'URL de la page présentant les résultats de la recherche
Cela se fait très facilement à partir du code suivant.

In [9]:
url = browser.current_url
print(url)

https://www.marmiton.org/recettes/recherche.aspx?aqt=farine


## 4.3. On récupère les url des pages correspondant aux différentes recettes trouvées

### 4.3.1 Grâce au xpahth, on récupère les url des recettes trouvées
a) On importe le package urllib3\
b) On installe la bibliothèque lxml qui est une bibliothèque de traitement prenant notamment en charge xpath\
c) On récupère le xpath de la page de résultats obtenue après la recherche précédente\
d) Cela nous permet, pour chacun des noeuds dépendant de ce xpath, de récupérer l'url correspondante\
e) On affiche les url des noeuds qui nous intéressent, c'est-à-dire de ceux menant à des recettes (et dont l'url commence donc par "/recettes/recette").

In [10]:
import urllib3
http = urllib3.PoolManager()
r= http.request('GET', url)
# si on veut : print(r.data)

from lxml import html

data_string = r.data.decode('utf-8', errors = 'ignore')
tree = html.fromstring(data_string)

links = tree.xpath('//a')
for link in links:
    l = link.get('href')
    if l.startswith('/recettes/recette'):
        print(l)

/recettes/recettes-incontournables.aspx
/recettes/recette_gateau-moelleux-a-la-farine-de-chataigne_85864.aspx
/recettes/recette_tortillas-recette-sans-farine-de-mais_40389.aspx
/recettes/recette_gateau-au-chocolat-sans-farine-et-sans-beurre_17669.aspx
/recettes/recette_cake-a-la-farine-de-chataigne_13766.aspx
/recettes/recette_moelleux-au-chocolat-sans-gluten-farine-de-riz_42417.aspx
/recettes/recette_besan-paratha-beignets-a-la-farine-de-pois-chiches-inde_15796.aspx
/recettes/recette_gateau-de-farine-de-chataigne-noisettes-et-chocolat_17184.aspx
/recettes/recette_cake-farine-complete-jambon-gruyere_37981.aspx
/recettes/recette_gateau-aux-pommes-a-la-farine-de-chataigne_54688.aspx
/recettes/recette_gateau-au-chocolat-fondant-sans-farine_30347.aspx
/recettes/recette_gateau-au-yaourt-aux-3-farines_42407.aspx
/recettes/recette_gateau-a-l-orange-et-poudre-d-amande-sans-farine_371165.aspx


### 4.3.2. On crée une liste avec les url correspondant aux recettes trouvées
Pour cela, on commence par créer une liste vide. Pour chaque noeud du xpath trouvé précédemment, on obtient l'url grâce à la méthode "get('href')". On ne conserve que les noeuds menant à des recettes, c'est-à-dire ceux dont l'url commence par "/recettes/recette". Enfin, pour que le navigateur puisse suivre ces url par la suite, il faut ajouter "https://www.marmiton.org".\
On obtient alors une liste d'url pertinentes et fonctionnelles. 

In [11]:
liste_url_recettes = []
for link in links:
    l = link.get('href')
    if l.startswith('/recettes/recette'):
        url_recette = 'https://www.marmiton.org'+l
        liste_url_recettes.append(url_recette)
print(liste_url_recettes)

['https://www.marmiton.org/recettes/recettes-incontournables.aspx', 'https://www.marmiton.org/recettes/recette_gateau-moelleux-a-la-farine-de-chataigne_85864.aspx', 'https://www.marmiton.org/recettes/recette_tortillas-recette-sans-farine-de-mais_40389.aspx', 'https://www.marmiton.org/recettes/recette_gateau-au-chocolat-sans-farine-et-sans-beurre_17669.aspx', 'https://www.marmiton.org/recettes/recette_cake-a-la-farine-de-chataigne_13766.aspx', 'https://www.marmiton.org/recettes/recette_moelleux-au-chocolat-sans-gluten-farine-de-riz_42417.aspx', 'https://www.marmiton.org/recettes/recette_besan-paratha-beignets-a-la-farine-de-pois-chiches-inde_15796.aspx', 'https://www.marmiton.org/recettes/recette_gateau-de-farine-de-chataigne-noisettes-et-chocolat_17184.aspx', 'https://www.marmiton.org/recettes/recette_cake-farine-complete-jambon-gruyere_37981.aspx', 'https://www.marmiton.org/recettes/recette_gateau-aux-pommes-a-la-farine-de-chataigne_54688.aspx', 'https://www.marmiton.org/recettes/rece

## 4.4. Visualisation
Cette liste d'url nous permet de retrouver les différentes recettes en suivant le lien et également de trouver leurs noms, qui sont à chaque fois contenus dans l'url.

### 4.4.1 Pour chaque recette, on peut suivre l'url

In [12]:
import requests

for recette in liste_url_recettes:
    print(requests.get(recette))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


### 4.4.2. On peut afficher les titres des différentes recettes
En effet les url des recettes sont constuits selon un format semblable, ce qui fait que le nom de la recette est toujours un segment de l'url situé à la même position. Il suffit donc de ne choisir que certains caractères de l'url.

In [13]:
for recette in liste_url_recettes:
    print(recette[42:-11])

-incontour
gateau-moelleux-a-la-farine-de-chataigne
tortillas-recette-sans-farine-de-mais
gateau-au-chocolat-sans-farine-et-sans-beurre
cake-a-la-farine-de-chataigne
moelleux-au-chocolat-sans-gluten-farine-de-riz
besan-paratha-beignets-a-la-farine-de-pois-chiches-inde
gateau-de-farine-de-chataigne-noisettes-et-chocolat
cake-farine-complete-jambon-gruyere
gateau-aux-pommes-a-la-farine-de-chataigne
gateau-au-chocolat-fondant-sans-farine
gateau-au-yaourt-aux-3-farines
gateau-a-l-orange-et-poudre-d-amande-sans-farine_


## 4.5. On crée une liste de recettes
Chaque recette est un dictionnaire comportant le nom de la recette, la liste des ingrédients et l'url associé.\
a) On commence par créer une liste vide\
b) Pour chaque recette (à partir de l'url trouvé plus haut), on définit le nom de la recette\
c) Pour chaque recette, grâce à une recherche dans le code source, on trouve la liste des ingrédients\
d) Pour chaque recette, on crée un dictionnaire avec le nom de la recette, la liste des ingrédients et l'url renvoyant à la page marmiton correspondante\
e) Enfin, on enlève la première "recette", qui correspond, non pas à une véritable recette mais à une page de "recettes incontournables" qui ne nous intéressent pas ici

In [14]:
liste_recettes = []

for url_recette in liste_url_recettes:
    
    titre = url_recette[42:-11]
    
    browser.get(url_recette)
    liste_url_ingredients = browser.find_elements(By.CLASS_NAME, 'card-ingredient')
    liste_ingredients = []
    for x in liste_url_ingredients:
        liste_ingredients.append(x.text)

    recette = {'Recette':titre, 'Liste des ingrédients':liste_ingredients, 'Lien vers la recette':url_recette}
    
    liste_recettes.append(recette)
    
print(liste_recettes[1:])

[{'Recette': 'gateau-moelleux-a-la-farine-de-chataigne', 'Liste des ingrédients': ['1 pincée\nde sel', "1 bonne cuillère à café\nd' extrait de vanille liquide", '1 sachet\nde levure chimique', "10 cl\nd' huile", '180 g\nde sucre blanc ou sucre de canne', '160 g\nde farine de châtaigne', '10 cl\nde lait', '4\noeufs', "5 cl\nd' eau de vie ou de rhum"], 'Lien vers la recette': 'https://www.marmiton.org/recettes/recette_gateau-moelleux-a-la-farine-de-chataigne_85864.aspx'}, {'Recette': 'tortillas-recette-sans-farine-de-mais', 'Liste des ingrédients': ['1 cuillère à café rase\nde sel', "5 cl\nd' huile de tournesol", '250 g\nde farine de blé (+ 1 ou 2 cuillère pour le plan de travail)', "13 cl\nd' eau tiède salée"], 'Lien vers la recette': 'https://www.marmiton.org/recettes/recette_tortillas-recette-sans-farine-de-mais_40389.aspx'}, {'Recette': 'gateau-au-chocolat-sans-farine-et-sans-beurre', 'Liste des ingrédients': ['100 g\nde sucre', '2 tablettes\nde chocolat à croquer', '4\noeufs', "100 

# **5. Fin du webscrapping : on quitte la recherche**
Le webscrapping est terminé, on peut donc quitter le navigateur.

In [ ]:
browser.quit()